In [1]:
import pandas as pd
import streamlit as st
import plotly.express as px
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
import mlxtend.frequent_patterns as fp
import sklearn

In [2]:
columnas2 = ["ORIGEN","SECTOR","ENTIDAD_UM","SEXO","ENTIDAD_NAC","ENTIDAD_RES",
            "MUNICIPIO_RES","TIPO_PACIENTE","FECHA_INGRESO","INTUBADO","NEUMONIA","EDAD",
            "NACIONALIDAD","EMBARAZO","HABLA_LENGUA_INDIG","INDIGENA","DIABETES","EPOC","ASMA","INMUSUPR",
            "HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","OTRO_CASO",
             "TOMA_MUESTRA_LAB","RESULTADO_LAB","TOMA_MUESTRA_ANTIGENO", "RESULTADO_ANTIGENO","CLASIFICACION_FINAL",
            "UCI"]
datos_hoy = pd.read_csv("210124COVID19MEXICO.csv", usecols = columnas2)
datos_hoy['FECHA_INGRESO'] = pd.to_datetime(datos_hoy['FECHA_INGRESO'])
X = datos_hoy[(datos_hoy['FECHA_INGRESO'] > '2020-12-20 00:00:00')]
X.drop(['FECHA_INGRESO'], axis=1,inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [3]:
Mor = X[X["ENTIDAD_UM"]==16]
CM = X[X["ENTIDAD_UM"]==9]

2021-02-07 16:48:05.684 INFO    numexpr.utils: NumExpr defaulting to 4 threads.


# Preprocessing data

Not using PAIS_NACIONALIDAD as most of the data is from Mexico and may bias clustering analysis. 

In [4]:
estado_casos={ 1:"AGUASCALIENTES",2:"BAJA CALIFORNIA",3:"BAJA CALIFORNIA SUR",4:"CAMPECHE",5:"COAHUILA DE ZARAGOZA",6:"COLIMA",7:"CHIAPAS",8:"CHIHUAHUA",9:"CIUDAD DE MÉXICO",
            10:"DURANGO",11:"GUANAJUATO",12:"GUERRERO",13:"HIDALGO",14:"JALISCO",15:"MÉXICO",16:"MICHOACÁN DE OCAMPO",17:"MORELOS",18:"NAYARIT",19:"NUEVO LEÓN",
            20:"OAXACA",21:"PUEBLA",22:"QUERÉTARO",23:"QUINTANA ROO",24:"SAN LUIS POTOSÍ",25:"SINALOA",26:"SONORA", 27:"TABASCO",28:"TAMAULIPAS",29:"TLAXCALA",
            30:"VERACRUZ DE IGNACIO DE LA LLAVE",31:"YUCATÁN",32:"ZACATECAS",99:'No'}

In [4]:
pd.DataFrame(lista.values(), index=lista.keys(), columns=["Estado","Casos"]).to_csv("CasosMex.csv")

NameError: name 'lista' is not defined

In [61]:
lista

{9: ['CIUDAD DE MÉXICO', 1334361],
 15: ['MÉXICO', 463383],
 3: ['BAJA CALIFORNIA SUR', 38080],
 14: ['JALISCO', 155146],
 23: ['QUINTANA ROO', 20211],
 17: ['MORELOS', 44378],
 11: ['GUANAJUATO', 220726],
 26: ['SONORA', 102719],
 21: ['PUEBLA', 149269],
 29: ['TLAXCALA', 42951],
 20: ['OAXACA', 85061],
 31: ['YUCATÁN', 76280],
 2: ['BAJA CALIFORNIA', 59568],
 25: ['SINALOA', 86419],
 16: ['MICHOACÁN DE OCAMPO', 108594],
 27: ['TABASCO', 142179],
 8: ['CHIHUAHUA', 65498],
 4: ['CAMPECHE', 24921],
 7: ['CHIAPAS', 64579],
 1: ['AGUASCALIENTES', 56906],
 5: ['COAHUILA DE ZARAGOZA', 134624],
 19: ['NUEVO LEÓN', 182046],
 30: ['VERACRUZ DE IGNACIO DE LA LLAVE', 147428],
 12: ['GUERRERO', 85338],
 28: ['TAMAULIPAS', 109621],
 32: ['ZACATECAS', 56007],
 24: ['SAN LUIS POTOSÍ', 116490],
 10: ['DURANGO', 69575],
 22: ['QUERÉTARO', 56178],
 6: ['COLIMA', 16229],
 18: ['NAYARIT', 27312],
 13: ['HIDALGO', 63780]}

In [64]:
coord = [[19.380866,-99.121148],[19.371397,-99.672654],[25.617409,-111.679666],[20.493409,-103.486139],
         [19.263021,-87.450912],[18.681305,-99.101349],[20.863436,-101.155482],[29.098901,-109.950413],
         [19.123424,-98.214963],[19.419973,-98.112054],[],[20.400417,-89.134857],
         [16.746805,-96.591216],[],[19.187134,-101.885976],
         [],[],[],[],[],[18.020529,-99.662476],[],[],[],[],[],[],[],[],[],[],[]]

In [116]:
afectaciones

{'NEUMONIA': [402011, 4006642, 420026],
 'EMBARAZO': [33800, 2245455, 2132212, 2294456],
 'DIABETES': [471476, 3943804, 482864],
 'EPOC': [44552, 4371727, 54941],
 'ASMA': [111422, 4305050, 121618],
 'INMUSUPR': [40449, 4375613, 51055],
 'HIPERTENSION': [635637, 3780552, 646116],
 'OTRO_CASO': [2028597, 2097377, 2329291],
 'CARDIOVASCULAR': [66490, 4349852, 76816],
 'OBESIDAD': [563072, 3853606, 573062],
 'TABAQUISMO': [364251, 4051560, 375108],
 'RENAL_CRONICA': [60991, 4355491, 71177]}

In [117]:
pd.DataFrame(afectaciones.values(), index = afectaciones.keys())

,0,1,2,3
NEUMONIA,402011,4006642,420026,NaN
EMBARAZO,33800,2245455,2132212,2294456.0
DIABETES,471476,3943804,482864,NaN
EPOC,44552,4371727,54941,NaN
ASMA,111422,4305050,121618,NaN
INMUSUPR,40449,4375613,51055,NaN
HIPERTENSION,635637,3780552,646116,NaN
OTRO_CASO,2028597,2097377,2329291,NaN
CARDIOVASCULAR,66490,4349852,76816,NaN
OBESIDAD,563072,3853606,573062,NaN


In [5]:

ORIGEN = {1:"USMER",2:"Otro origen",99:"Sin especificar_origen"}
SECTOR = {1:"CRUZ ROJA",2:"DIF",3:'ESTATAL',4:'IMSS',5:'IMSS-BIENESTAR',6:'ISSTE',7:'MUNICIPAL',
          8:'PEMEX',9:'PRIVADA',10:'SEDEBA',11:'SEMAR',12:'SSA',13:'UNIVERSITARIO',99:'No especifica_sector'}
ENTIDAD_UM = {1:"AGUASCALIENTES",2:"BAJA CALIFORNIA",3:"BAJA CALIFORNIA SUR",4:"CAMPECHE",5:"COAHUILA",6:"COLIMA",
              7:"CHIAPAS",8:"CHIHUAHUA",9:"CIUDAD DE MEXICO",10:"DURANGO",11:"GUANAJUATO",12:"GUERRERO",
              13:"HIDALGO",14:"JALISCO",15:"ESTADO DE MEXICO",16:"MICHOACAN",17:"MORELOS",18:"NAYARIT",19:"NUEVO LEON",
              20:"OAXACA",21:"PUEBLA",22:"QUERETARO",23:"QUINTANA ROO",24:"SAN LUIS POTOSI",25:"SINALOA",
              26:"SONORA",27:"TABASCO",28:"TAMAULIPAS",29:"TLAXCALA",30:"VERACRUZ",31:"YUCATAN",32:"ZACATECAS",
              36:"OTRO",97:"EXTRANJERO",98:"OTRO",99:"SIN ESPECIFICAR"}
TIPO_PACIENTE= {1:"AMBULATORIO",2:"HOSPITALIZADO",99:"SIN ESPECIFICAR_TIPO_PACIENTE"}
SI_NO = {1:"SI",2:"NO",97:"No aplica",98:"Se ignora",99:"No especifica"}
SEXO = {1:"MUJER",2:"HOMBRE",3:"NO ESPECIFICA_SEXO"}
RESULTADO_LAB = {1:"POSITIVO",2:"NEGATIVO",3:"PENDIENTE",4:"PENDIENTE",97:"POSITIVO"}
RESULTADO_ANTIGENO = {1:"POSITIVO",2:"NEGATIVO",97:"NO ESPECIFICA_ANTIGENO"}
CLASIFICACION_FINAL = {1:"POSITIVO_CONTACTO",2:"POSITIVO_DEFUNCION",3:"POSITIVO_OTRO",4:"PENDIENTE_MUESTRA",
                      5:"PENDIENTE_LABORATORIO",6:"PENDIENTE_SOSPECHA",7:"NEFGATIVO"}

SINO = ["INTUBADO","NEUMONIA","EMBARAZO","HABLA_LENGUA_INDIG","INDIGENA","DIABETES","EPOC","ASMA","INMUSUPR",
       "HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","OTRO_CASO",
       "TOMA_MUESTRA_LAB","TOMA_MUESTRA_ANTIGENO","UCI"]
dic ={ "ORIGEN":[],"SECTOR":[],"ENTIDAD_UM":[],"SEXO":[],"ENTIDAD_NAC":[],
            "TIPO_PACIENTE":[],"INTUBADO":[],"NEUMONIA":[],"EDAD":[],
            "EMBARAZO":[],"HABLA_LENGUA_INDIG":[],"INDIGENA":[],"DIABETES":[],"EPOC":[],"ASMA":[],"INMUSUPR":[],
            "HIPERTENSION":[],"OTRA_COM":[],"CARDIOVASCULAR":[],"OBESIDAD":[],"RENAL_CRONICA":[],
             "TABAQUISMO":[],"OTRO_CASO":[],
             "TOMA_MUESTRA_LAB":[],"RESULTADO_LAB":[],"TOMA_MUESTRA_ANTIGENO":[], "RESULTADO_ANTIGENO":[],
      "CLASIFICACION_FINAL":[],
    "UCI":[]}

In [6]:
for registro in X["ORIGEN"]:
    dic["ORIGEN"].append(ORIGEN[registro]) 
for registro in X["SECTOR"]:
    dic["SECTOR"].append(SECTOR[registro])
for registro in X["ENTIDAD_UM"]:
    dic["ENTIDAD_UM"].append(ENTIDAD_UM[registro])
for registro in X["SEXO"]:
    dic["SEXO"].append(SEXO[registro])
for registro in X["ENTIDAD_NAC"]:
    dic["ENTIDAD_NAC"].append(ENTIDAD_UM[registro])
for registro in X["TIPO_PACIENTE"]:
    dic["TIPO_PACIENTE"].append(TIPO_PACIENTE[registro])
for registro in X["INTUBADO"]:
    dic["INTUBADO"].append(SI_NO[registro]+"INTUBADO")
for registro in X["NEUMONIA"]:
    dic["NEUMONIA"].append(SI_NO[registro]+"NEUMONIA")
for registro in X["EDAD"]:
    dic["EDAD"].append(str(registro))
for registro in X["EMBARAZO"]:
    dic["EMBARAZO"].append(SI_NO[registro]+"EMBARAZO")
for registro in X["HABLA_LENGUA_INDIG"]:
    dic["HABLA_LENGUA_INDIG"].append(SI_NO[registro]+"LENGUA")
for registro in X["INDIGENA"]:
    dic["INDIGENA"].append(SI_NO[registro]+"INDIGENA")
for registro in X["DIABETES"]:
    dic["DIABETES"].append(SI_NO[registro]+"DIABETES")
for registro in X["EPOC"]:
    dic["EPOC"].append(SI_NO[registro]+"EPOC")
for registro in X["ASMA"]:
    dic["ASMA"].append(SI_NO[registro]+"ASMA")
for registro in X["INMUSUPR"]:
    dic["INMUSUPR"].append(SI_NO[registro]+"INMU")
for registro in X["HIPERTENSION"]:
    dic["HIPERTENSION"].append(SI_NO[registro]+"HIPER")
for registro in X["OTRA_COM"]:
    dic["OTRA_COM"].append(SI_NO[registro]+"OTRAS_COM")
for registro in X["CARDIOVASCULAR"]:
    dic["CARDIOVASCULAR"].append(SI_NO[registro]+"CARDIO")
for registro in X["OBESIDAD"]:
    dic["OBESIDAD"].append(SI_NO[registro]+"OBESIDAD")
for registro in X["RENAL_CRONICA"]:
    if registro in SI_NO.keys():
        dic["RENAL_CRONICA"].append(SI_NO[registro]+"RENAL")
    else:
        dic["RENAL_CRONICA"].append(SI_NO[97]+"RENAL")
for registro in X["TABAQUISMO"]:
    if registro in SI_NO.keys():
        dic["TABAQUISMO"].append(SI_NO[registro]+"TABACO")
    else:
        dic["TABAQUISMO"].append(SI_NO[97]+"TABACO")
for registro in X["OTRO_CASO"]:
    if registro in SI_NO.keys():
        dic["OTRO_CASO"].append(SI_NO[registro]+"OTRO_CASO")
    else:
        dic["OTRO_CASO"].append(SI_NO[97])
for registro in X["TOMA_MUESTRA_LAB"]:
    if registro in SI_NO.keys():
        dic["TOMA_MUESTRA_LAB"].append(SI_NO[registro]+"TOMA_LAB")
    else:
        dic["TOMA_MUESTRA_LAB"].append(SI_NO[97]+"TOMA_LAB")
for registro in X["RESULTADO_LAB"]:
    if registro in RESULTADO_LAB.keys():
        dic["RESULTADO_LAB"].append(RESULTADO_LAB[registro])
    else:
        dic["RESULTADO_LAB"].append(RESULTADO_LAB[3])
for registro in X["TOMA_MUESTRA_ANTIGENO"]:
    if registro in SI_NO.keys():
        dic["TOMA_MUESTRA_ANTIGENO"].append(SI_NO[registro]+"TOMA_ANT")
    else:
        dic["TOMA_MUESTRA_ANTIGENO"].append(SI_NO[97])
for registro in X["RESULTADO_ANTIGENO"]:
    if registro in RESULTADO_ANTIGENO.keys():
        dic["RESULTADO_ANTIGENO"].append(RESULTADO_ANTIGENO[registro])
    else:
        dic["RESULTADO_ANTIGENO"].append(RESULTADO_ANTIGENO[97])
for registro in X["CLASIFICACION_FINAL"]:
    if registro in CLASIFICACION_FINAL.keys():
        dic["CLASIFICACION_FINAL"].append(CLASIFICACION_FINAL[registro])
    else:
        dic["CLASIFICACION_FINAL"].append(CLASIFICACION_FINAL[6])

In [7]:
for registro in X["UCI"]:
    if registro in SI_NO.keys():
        dic["UCI"].append(SI_NO[registro]+"UCI")
    else:
        dic["UCI"].append(SI_NO[97])

In [8]:
deco = pd.DataFrame(dic)

In [9]:
deco_mor = deco[deco["ENTIDAD_UM"]=="MICHOACAN"]
deco_cm = deco[deco["ENTIDAD_UM"]=="CIUDAD DE MEXICO"]

# Encoding

In [10]:
encoder_ = TransactionEncoder()
encoder_mor = TransactionEncoder()
encoder_cm = TransactionEncoder()

In [11]:
Coded =encoder_.fit(deco.to_numpy()).transform(deco.to_numpy())

In [12]:
Coded_mor = encoder_mor.fit(deco_mor.to_numpy()).transform(deco_mor.to_numpy())

In [13]:
Coded_cm = encoder_cm.fit(deco_cm.to_numpy()).transform(deco_cm.to_numpy())

In [14]:
datos = pd.DataFrame(Coded,columns=encoder_.columns_)
datos_mor = pd.DataFrame(Coded_mor,columns=encoder_mor.columns_)
datos_cm = pd.DataFrame(Coded_cm,columns=encoder_cm.columns_)

In [15]:
import mlxtend.frequent_patterns as FP
import numpy as np

In [16]:
PF_mor = FP.apriori(datos_mor,min_support=0.5, use_colnames=True)
PF_mor

,support,itemsets
0,0.789888,(AMBULATORIO)
1,1.000000,(MICHOACAN)
2,0.522937,(MUJER)
3,0.789134,(NO ESPECIFICA_ANTIGENO)
4,0.972627,(NOASMA)
...,...,...
500970,0.501088,"(NOHIPER, NOOBESIDAD, NONEUMONIA, NOTABACO, No..."
500971,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA..."
500972,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA..."
500973,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA..."


In [18]:
PF_mor['k'] = PF_mor['itemsets'].apply(lambda x:len(x))
PF_mor

,support,itemsets,k
0,0.789888,(AMBULATORIO),1
1,1.000000,(MICHOACAN),1
2,0.522937,(MUJER),1
3,0.789134,(NO ESPECIFICA_ANTIGENO),1
4,0.972627,(NOASMA),1
...,...,...,...
500970,0.501088,"(NOHIPER, NOOBESIDAD, NONEUMONIA, NOTABACO, No...",16
500971,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA...",16
500972,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA...",16
500973,0.500335,"(NOTABACO, NONEUMONIA, No aplicaUCI, SITOMA_LA...",16


In [23]:
PF_mor=PF_mor[PF_mor['k']>=10]

In [36]:
PF_mor = PF_mor[PF_mor["support"]>=0.70]

In [43]:
PF_mor

,support,itemsets,k
405142,0.760589,"(NODIABETES, NOLENGUA, NOCARDIO, NOOTRAS_COM, ...",10
407246,0.754395,"(NONEUMONIA, NOLENGUA, NOCARDIO, NOOTRAS_COM, ...",10
407296,0.801691,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",10


In [44]:
PF_mor.to_csv("FrecuentesMorelia.csv")

In [41]:
reglas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",(NOINDIGENA),NaN,NaN,0.801691,NaN,NaN,NaN,NaN
1,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",(NORENAL),NaN,NaN,0.801691,NaN,NaN,NaN,NaN
2,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",(MICHOACAN),NaN,NaN,0.801691,NaN,NaN,NaN,NaN
3,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",(NOASMA),NaN,NaN,0.801691,NaN,NaN,NaN,NaN
4,"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",(NOINMU),NaN,NaN,0.801691,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1017,(NOINMU),"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",NaN,NaN,0.801691,NaN,NaN,NaN,NaN
1018,(NOASMA),"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",NaN,NaN,0.801691,NaN,NaN,NaN,NaN
1019,(MICHOACAN),"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",NaN,NaN,0.801691,NaN,NaN,NaN,NaN
1020,(NORENAL),"(NOTABACO, NOLENGUA, NOCARDIO, NOOTRAS_COM, NO...",NaN,NaN,0.801691,NaN,NaN,NaN,NaN
